In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt; 
 
# Importing sklearn libraries
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score
 

 
from keras.utils import to_categorical #to create dummy variable
from keras.utils import np_utils
from keras.models import Sequential
from keras.applications import VGG16
from keras.applications import imagenet_utils
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.layers import Dense, Conv2D, MaxPooling2D
from keras.layers import Dropout, Flatten, GlobalAveragePooling2D


import import_ipynb
import utils



from sklearn.svm import SVC

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# SOME IMPORTANT GLOBAL VARIABLE
labels = ["Bedroom","Coast","Forest","HighWay","Industrial","InsideCity","Kitchen","LivingRoom","Mountain","Office","OpenCountry","Store","Street","Suburb","TallBuilding"]
train_dir = '../images/train/'
test_dir = '../images/test/'


# CREATION OF DATA--> test and train 


In [ ]:
#LIST OF ISTRING WHICH REPRESENT ALL THE IMAGES!
list_of_images = utils.list_of_path(labels,train_dir)
train_data,train_labels = utils.read_and_process_images(list_of_images,dimension=224)

train_labels_dummy  = to_categorical(train_labels,15)

In [ ]:
list_of_images_test = utils.list_of_path(labels,test_dir)
test_data,test_labels = utils.read_and_process_images(list_of_images_test,dimension=224)

test_labels_dummy  = to_categorical(test_labels,15)


#  model choice (VGG16) and feature extraction 

In [ ]:
model = VGG16(weights='imagenet', include_top=False)


In [ ]:
model.summary()

In [ ]:
imagePath =  list_of_images[1]
image = load_img(imagePath, target_size=(224, 224))
image = img_to_array(image)
image = np.expand_dims(image, axis=0)
image = imagenet_utils.preprocess_input(image)

In [ ]:
image.shape

In [ ]:
# define a function which extract feature for all my train images

def feature_extraction(dataset,pre_model):
    res = []
    cont = 0
    #viaggio nel dataset
    for path in dataset:
        print(cont)
        im = load_img(path, target_size=(224, 224))
        im = img_to_array(im)
        im = np.expand_dims(im, axis=0)
        im = imagenet_utils.preprocess_input(im)
        res.append(im)
    images = np.vstack(res)
    print("inizio")
    features = pre_model.predict(images, batch_size=64)
    print("fine")
    features_flatten = features.reshape((features.shape[0], 7 * 7 * 512))
    return images, features, features_flatten     

In [ ]:
train_images,feature_matrix,feature_array= feature_extraction(list_of_images,model)

In [ ]:
test_images,feature_matrix_test,feature_array_test = feature_extraction(list_of_images_test,model)

In [ ]:
def predict_with_linear_SVM(features_array,feature_lab,feature_array_test,test_labels,num_classes = 15):
    classif = [SVC(kernel="linear") for _ in range(num_classes)] # array of classifier to be trained
    # one-vs-all approach with  
    curr_label = 0
    for clf in classif:
        print(curr_label)
        v = np.array([1 if label==curr_label else 0 for label in feature_lab])
        clf = clf.fit(features_array, v)
        curr_label = curr_label + 1
    # now we want to test 
    prediction = []
    for image in feature_array_test:
        pred = np.array([np.dot(clf.coef_,image) + clf.intercept_ for clf in classif])
        prediction.append(np.argmax(pred))
    prediction = np.asarray(prediction)
    #calculate accuracy
    cont=0
    for i in range(len(prediction)):
        if prediction[i]==test_labels[i]:
            cont = cont +1
        else:
            continue
        
    acc = cont/len(prediction)
    
    return classif,prediction,acc
                    
    
    

In [ ]:
classifiers,prediction,accuracy = predict_with_linear_SVM(feature_array,train_labels,feature_array_test,test_labels)


In [ ]:
accuracy

# UTILIZZO DAG PER IL MULTICLASS SVM 


In [ ]:
def predict_with_SVM_with_Dag(features_array,feature_lab,feature_array_test,test_labels,num_classes = 15):
    #creare dictionary dove ad ogni indice (relativo a test_labels) associo il test_labels
    res_dict = {}
    test_dict = {}
    for i in range(len(test_labels)):
        test_dict[i]=test_labels[i]
    # creo dictionary che fa la stessa cosa per il train
    train_dict={}
    for i in range(len(feature_lab)):
        train_dict[i]=feature_lab[i]
    n_class = num_classes - 1
    for curr_label in range(n_class):
        if(curr_label<n_class-1):
            clf = SVC(kernel = "linear")
            v = np.array([1 if label==curr_label else 0 for label in list(test_dict.values())])
            clf = clf.fit(np.asarray(train_dict.values()), v)
            #ora faccio la predizione
            preds = clf.predict(np.asarray(list(test_dict.keys())))
            for i in range(len(list(test_dict.keys()))):
                if(preds[i]==1): #associato la predizione
                    res_dict[i]=curr_label
                    
                else:
                    continue
            # rimuovo elementi che sono stati assegnati da test_dict e da train_dict
            for keys in res_dict.keys():
                del test_dict[key]
                del train_dict[key]
        if(curr_label==n_class):
            clf = SVC(kernel = "linear")
            v = np.array([1 if label==curr_label else 0 for label in list(test_dict.values())])
            clf = clf.fit(np.asarray(list(train_dict.values()), v))
            preds = clf.predict(np.asarray(list(test_dict.keys())))
            for i in range(len(list(test_dict.keys()))):
                if(preds[i]==1): #associato la predizione
                    res_dict[i]=curr_label
                    
                else:
                    res_dict[i]=curr_label+1
    
    return res_dict

        
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    
    